In [ ]:

import os, cv2, numpy as np, pandas as pd, torch, torch.nn as nn
import torchvision.transforms as T
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils.class_weight import compute_class_weight
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
from PIL import Image
from tqdm import tqdm

BASE1 = "/kaggle/input/soil-classification-2/soil_competition-2025/"
BASE2 = "/kaggle/input/soil-classification/soil_classification-2025/"
TR_IMG1 = os.path.join(BASE1, "train")
TR_IMG2 = os.path.join(BASE2, "train")
TE_IMG  = os.path.join(BASE1, "test")
LBL_CSV1= os.path.join(BASE1, "train_labels.csv")
LBL_CSV2= os.path.join(BASE2, "train_labels.csv")
IDS_CSV = os.path.join(BASE1, "test_ids.csv")

def smart_crop(img, thr=15):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    _, mask = cv2.threshold(gray, thr, 255, cv2.THRESH_BINARY)
    cnts, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not cnts: return img
    x, y, w, h = cv2.boundingRect(max(cnts, key=cv2.contourArea))
    return img[y:y+h, x:x+w]

df1 = pd.read_csv(LBL_CSV1).drop_duplicates("image_id")
df2 = pd.read_csv(LBL_CSV2).drop_duplicates("image_id")

df2["label"] = 1  # treat all multi‑class soil as soil

df1["path"] = df1["image_id"].apply(lambda x: os.path.join(TR_IMG1, x))
df2["path"] = df2["image_id"].apply(lambda x: os.path.join(TR_IMG2, x))

df = pd.concat([df1, df2], ignore_index=True)

if df["label"].nunique()==2:
    cw = compute_class_weight("balanced", classes=np.array([0,1]), y=df.label)
    class_wt = torch.tensor(cw, dtype=torch.float32)
else:
    class_wt = None

class SoilDataset(Dataset):
    def __init__(self, frame, tfm):
        self.df = frame.reset_index(drop=True)
        self.tfm = tfm
    def __len__(self): return len(self.df)
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = cv2.imread(row.path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = smart_crop(img)
        try:
            img = Image.fromarray(img)
            img = self.tfm(img)
        except Exception:
            fallback = Image.fromarray(np.zeros((256,256,3), dtype=np.uint8), mode="RGB")
            img = self.tfm(fallback)
        label = int(row.label) if "label" in row else -1
        return img, label, os.path.basename(row.path)

train_tf = T.Compose([
    T.Resize((256,256)),
    T.RandomAffine(degrees=15, translate=(0.1,0.1), scale=(0.9,1.1)),
    T.RandomHorizontalFlip(),
    T.RandomVerticalFlip(),
    T.ColorJitter(0.3,0.3,0.2,0.05),
    T.ToTensor(),
    T.RandomErasing(p=0.2, scale=(0.01,0.05), value=0),
    T.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

val_tf = T.Compose([
    T.Resize((256,256)),
    T.ToTensor(),
    T.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def train_one_model(rnd):
    print(f"\n🔁 Boost Round {rnd+1}")
    tr_df, val_df = train_test_split(df, test_size=0.2, stratify=df.label, random_state=42+rnd)
    tr_ld = DataLoader(
        SoilDataset(tr_df, train_tf),
        batch_size=32, shuffle=True, num_workers=2
    )
    vl_ld = DataLoader(
        SoilDataset(val_df, val_tf),
        batch_size=32, shuffle=False, num_workers=2
    )

    model = efficientnet_b0(weights=EfficientNet_B0_Weights.DEFAULT)
    model.classifier[1] = nn.Linear(model.classifier[1].in_features, 2)
    model = model.to(DEVICE)

    loss_fn = nn.CrossEntropyLoss(weight=class_wt.to(DEVICE) if class_wt is not None else None)
    opt = torch.optim.Adam(model.parameters(), 1e-4)
    scaler = torch.amp.GradScaler(enabled=DEVICE.type=="cuda")

    best = 0
    for ep in range(15):
        model.train(); tot=0
        for x,y,_ in tqdm(tr_ld, leave=False):
            x,y = x.to(DEVICE), y.to(DEVICE)
            opt.zero_grad()
            with torch.amp.autocast(device_type=DEVICE.type):
                loss = loss_fn(model(x), y)
            scaler.scale(loss).backward(); scaler.step(opt); scaler.update()
            tot += loss.item()
        # validation
        model.eval(); pr, gt = [],[]
        with torch.no_grad():
            for x,y,_ in vl_ld:
                out = model(x.to(DEVICE))
                pr += out.argmax(1).cpu().tolist(); gt += y.tolist()
        f1 = f1_score(gt, pr)
        print(f"Epoch {ep+1:02d} | loss {tot/len(tr_ld):.4f} | F1 {f1:.4f}")
        if f1>best:
            best=f1; torch.save(model.state_dict(), f"/kaggle/working/best_{rnd}.pth")
    return model

# ------------ Ensemble Train ----------
models=[]
for r in range(3):
    m=train_one_model(r)
    m.load_state_dict(torch.load(f"/kaggle/working/best_{r}.pth")); m.eval(); models.append(m)

# -------------- Inference -------------
test_df = pd.read_csv(IDS_CSV)
test_df["path"] = test_df.image_id.apply(lambda x: os.path.join(TE_IMG,x))

test_ld = DataLoader(
    SoilDataset(test_df, val_tf),
    batch_size=32, shuffle=False, num_workers=2
)
pr_all, ids = [], []
with torch.no_grad():
    for x,_,nm in tqdm(test_ld):
        x=x.to(DEVICE)
        probs = sum(torch.softmax(m(x),1) for m in models)/len(models)
        pr_all += probs.argmax(1).cpu().tolist(); ids += nm

sub = pd.DataFrame({"image_id":ids,"label":pr_all})
sub = test_df.set_index("image_id").join(sub.set_index("image_id")).reset_index()
sub.to_csv("/kaggle/working/submission.csv", index=False)
print("✅ submission.csv saved!")
display(sub.head())
